In [ ]:
# ============================================================
# 🌀 WaterMarkEye - DWT-DCT Invisible Watermarking System
# ============================================================

import cv2
import numpy as np
import pywt
import matplotlib.pyplot as plt
import gradio as gr
from skimage.metrics import structural_similarity as ssim
import tempfile
import os

# -------------------- Core DWT-DCT Functions --------------------

def embed_watermark_dwt_dct(cover_img, watermark_binary, alpha=0.1):
    """
    Step 2-5: Preprocessing + DWT + DCT + Embedding + Reconstruction
    """
    # Convert to YCbCr
    cover_rgb = cv2.cvtColor(cover_img, cv2.COLOR_BGR2RGB)
    ycbcr = cv2.cvtColor(cover_rgb, cv2.COLOR_RGB2YCrCb)
    Y = np.float32(ycbcr[:,:,0])

    # Apply DWT (2-level for better robustness)
    coeffs2 = pywt.wavedec2(Y, 'haar', level=2)
    LL2, (LH2, HL2, HH2), (LH1, HL1, HH1) = coeffs2

    # Resize watermark to fit LL2 subband
    ll_h, ll_w = LL2.shape
    block_size = 8
    wm_resized = cv2.resize(watermark_binary,
                           (ll_w // block_size, ll_h // block_size),
                           interpolation=cv2.INTER_NEAREST)

    # Embed watermark in DCT coefficients
    LL2_modified = LL2.copy()
    wm_flat = wm_resized.flatten()
    idx = 0

    for i in range(0, ll_h - block_size + 1, block_size):
        for j in range(0, ll_w - block_size + 1, block_size):
            if idx >= len(wm_flat):
                break

            # Extract block
            block = LL2[i:i+block_size, j:j+block_size]

            # Apply DCT
            dct_block = cv2.dct(block)

            # Get watermark bit
            wm_bit = 1 if wm_flat[idx] > 0 else -1

            # Step 4: Embed in mid-frequency coefficients
            dct_block[3, 4] += alpha * wm_bit * np.std(dct_block)
            dct_block[4, 3] += alpha * wm_bit * np.std(dct_block)
            dct_block[2, 5] += alpha * wm_bit * np.std(dct_block)
            dct_block[5, 2] += alpha * wm_bit * np.std(dct_block)

            # Inverse DCT
            modified_block = cv2.idct(dct_block)
            LL2_modified[i:i+block_size, j:j+block_size] = modified_block
            idx += 1

    # Reconstruct with inverse DWT
    modified_coeffs = [LL2_modified, (LH2, HL2, HH2), (LH1, HL1, HH1)]
    Y_modified = pywt.waverec2(modified_coeffs, 'haar')

    # Handle dimension issues
    if Y_modified.shape != Y.shape:
        Y_modified = cv2.resize(Y_modified, (Y.shape[1], Y.shape[0]))

    # Reconstruct YCbCr
    ycbcr[:,:,0] = np.clip(Y_modified, 0, 255)
    watermarked_rgb = cv2.cvtColor(ycbcr, cv2.COLOR_YCrCb2RGB)

    # Store metadata
    metadata = {
        'original_watermark': watermark_binary,
        'wm_resized_shape': wm_resized.shape,
        'block_size': block_size,
        'alpha': alpha,
        'coeffs_shapes': [c.shape if isinstance(c, np.ndarray) else [s.shape for s in c]
                         for c in coeffs2]
    }

    return watermarked_rgb, metadata

def extract_watermark_dwt_dct(watermarked_img, metadata):
    """
    Step 6: Watermark detection using DWT-DCT correlation
    """
    # Get metadata
    original_wm = metadata['original_watermark']
    wm_resized_shape = metadata['wm_resized_shape']
    block_size = metadata['block_size']

    # Convert to YCbCr and get Y channel
    watermarked_rgb = cv2.cvtColor(watermarked_img, cv2.COLOR_BGR2RGB)
    ycbcr = cv2.cvtColor(watermarked_rgb, cv2.COLOR_RGB2YCrCb)
    Y = np.float32(ycbcr[:,:,0])

    # Apply 2-level DWT
    coeffs2 = pywt.wavedec2(Y, 'haar', level=2)
    LL2, (LH2, HL2, HH2), (LH1, HL1, HH1) = coeffs2

    # Extract watermark bits
    ll_h, ll_w = LL2.shape
    extracted_bits = []
    idx = 0
    total_bits = wm_resized_shape[0] * wm_resized_shape[1]

    for i in range(0, ll_h - block_size + 1, block_size):
        for j in range(0, ll_w - block_size + 1, block_size):
            if len(extracted_bits) >= total_bits:
                break

            # Extract block
            block = LL2[i:i+block_size, j:j+block_size]

            # Apply DCT
            dct_block = cv2.dct(block)

            # Extract from same coefficients
            val1 = dct_block[3, 4]
            val2 = dct_block[4, 3]
            val3 = dct_block[2, 5]
            val4 = dct_block[5, 2]

            # Calculate correlation
            avg_val = (val1 + val2 + val3 + val4) / 4.0

            # Use threshold for bit extraction
            bit = 1 if avg_val > 0 else 0
            extracted_bits.append(bit)
            idx += 1

    # Reshape and resize to original
    wm_extracted = np.array(extracted_bits[:total_bits]).reshape(wm_resized_shape)
    wm_extracted_resized = cv2.resize(wm_extracted,
                                     (original_wm.shape[1], original_wm.shape[0]),
                                     interpolation=cv2.INTER_NEAREST)

    return wm_extracted_resized

# -------------------- CORRECTED Attack Simulations --------------------

def jpeg_compress_attack(img, quality=75):
    """JPEG compression attack - CORRECTED color handling"""
    # Input is RGB, encode as JPEG and decode
    encode_param = [int(cv2.IMWRITE_JPEG_QUALITY), quality]
    _, encimg = cv2.imencode('.jpg', img, encode_param)
    decoded = cv2.imdecode(encimg, 1)
    return decoded  # Return in same color space (BGR)

def resize_attack(img, scale_factor=0.7):
    """Resizing attack - CORRECTED color handling"""
    h, w = img.shape[:2]
    new_h, new_w = int(h * scale_factor), int(w * scale_factor)
    resized = cv2.resize(img, (new_w, new_h))
    return cv2.resize(resized, (w, h))  # Maintain original color space

def gaussian_noise_attack(img, noise_level=25):
    """Add Gaussian noise to image - CORRECTED color handling"""
    # Apply noise to each channel separately in BGR space
    noise = np.random.normal(0, noise_level, img.shape).astype(np.float32)
    noisy_img = img.astype(np.float32) + noise
    return np.clip(noisy_img, 0, 255).astype(np.uint8)  # Maintain BGR

def brightness_attack(img, brightness_factor=1.3):
    """Adjust brightness of image - CORRECTED color handling"""
    # Convert to HSV for brightness adjustment
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    hsv = hsv.astype(np.float32)
    # Only modify Value (brightness) channel
    hsv[:,:,2] = hsv[:,:,2] * brightness_factor
    hsv[:,:,2] = np.clip(hsv[:,:,2], 0, 255)
    hsv = hsv.astype(np.uint8)
    # Convert back to BGR
    return cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)

def blur_attack(img, kernel_size=5):
    """Apply Gaussian blur to image - CORRECTED color handling"""
    # Apply blur directly in BGR space
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

def contrast_attack(img, contrast_factor=1.5):
    """Adjust contrast of image - CORRECTED color handling"""
    # Apply contrast in BGR space using alpha blending
    return cv2.convertScaleAbs(img, alpha=contrast_factor, beta=0)

# -------------------- Gradio Interface Functions --------------------

class WatermarkSystem:
    def __init__(self):
        self.cover_image = None
        self.logo_image = None
        self.watermarked_image = None
        self.metadata = None
        self.logo_binary = None
        self.accuracy_results = {}

    def preprocess_logo(self, logo_img):
        """Convert logo to binary format"""
        logo_gray = cv2.cvtColor(logo_img, cv2.COLOR_BGR2GRAY)
        _, logo_binary = cv2.threshold(logo_gray, 127, 1, cv2.THRESH_BINARY)
        return logo_binary

    def calculate_similarity(self, original, extracted):
        """Calculate similarity between original and extracted watermarks"""
        if original.shape != extracted.shape:
            extracted = cv2.resize(extracted, (original.shape[1], original.shape[0]))

        similarity = ssim(original, extracted, data_range=extracted.max() - extracted.min())
        accuracy = np.mean(original == extracted)

        return similarity, accuracy

    def upload_cover(self, image):
        """Handle cover image upload"""
        if image is None:
            return None, "❌ Please upload a cover image"
        self.cover_image = image
        # Reset results when new cover is uploaded
        self.accuracy_results = {}
        return image, "✅ Cover image uploaded successfully!"

    def upload_logo(self, image):
        """Handle logo image upload"""
        if image is None:
            return None, "❌ Please upload a logo image"
        self.logo_image = image
        self.logo_binary = self.preprocess_logo(image)
        # Reset results when new logo is uploaded
        self.accuracy_results = {}

        # Display binary logo
        binary_display = (self.logo_binary * 255).astype(np.uint8)
        return image, "✅ Logo image uploaded successfully!"

    def embed_watermark(self):
        """Embed watermark into cover image"""
        if self.cover_image is None:
            return None, "❌ Please upload cover image first"
        if self.logo_image is None:
            return None, "❌ Please upload logo image first"

        try:
            self.watermarked_image, self.metadata = embed_watermark_dwt_dct(
                self.cover_image, self.logo_binary, alpha=0.15)

            # Convert back to BGR for display
            watermarked_bgr = cv2.cvtColor(self.watermarked_image, cv2.COLOR_RGB2BGR)
            return watermarked_bgr, "✅ Watermark embedded successfully!"
        except Exception as e:
            return None, f"❌ Error embedding watermark: {str(e)}"

    def extract_clean(self):
        """Extract watermark from clean watermarked image"""
        if self.watermarked_image is None:
            return None, "❌ Please embed watermark first"

        try:
            extracted_wm = extract_watermark_dwt_dct(
                cv2.cvtColor(self.watermarked_image, cv2.COLOR_RGB2BGR), self.metadata)

            # Convert to display format
            extracted_display = (extracted_wm * 255).astype(np.uint8)

            # Calculate accuracy
            similarity, accuracy = self.calculate_similarity(self.logo_binary, extracted_wm)

            # Store clean extraction result
            self.accuracy_results['Clean Extraction'] = accuracy

            return extracted_display, f"✅ Clean extraction completed! Accuracy: {accuracy:.1%}"
        except Exception as e:
            return None, f"❌ Error extracting watermark: {str(e)}"

    def apply_attack_and_extract(self, attack_type):
        """Apply specific attack and extract watermark"""
        if self.watermarked_image is None:
            return None, None, "❌ Please embed watermark first"

        try:
            # Convert watermarked image to BGR for attacks
            watermarked_bgr = cv2.cvtColor(self.watermarked_image, cv2.COLOR_RGB2BGR)

            # Apply attack
            if attack_type == "JPEG Compression":
                attacked_img = jpeg_compress_attack(watermarked_bgr, quality=70)
                attack_name = "JPEG Compression"
            elif attack_type == "Resizing":
                attacked_img = resize_attack(watermarked_bgr, scale_factor=0.6)
                attack_name = "Resizing"
            elif attack_type == "Gaussian Noise":
                attacked_img = gaussian_noise_attack(watermarked_bgr, noise_level=20)
                attack_name = "Gaussian Noise"
            elif attack_type == "Brightness":
                attacked_img = brightness_attack(watermarked_bgr, brightness_factor=1.4)
                attack_name = "Brightness"
            elif attack_type == "Blur":
                attacked_img = blur_attack(watermarked_bgr, kernel_size=5)
                attack_name = "Blur"
            elif attack_type == "Contrast":
                attacked_img = contrast_attack(watermarked_bgr, contrast_factor=1.5)
                attack_name = "Contrast"
            else:
                return None, None, "❌ Unknown attack type"

            # Extract watermark from attacked image
            extracted_wm = extract_watermark_dwt_dct(attacked_img, self.metadata)
            extracted_display = (extracted_wm * 255).astype(np.uint8)

            # Calculate accuracy
            similarity, accuracy = self.calculate_similarity(self.logo_binary, extracted_wm)

            # Store result
            self.accuracy_results[attack_name] = accuracy

            return attacked_img, extracted_display, f"✅ {attack_name} completed! Extraction Accuracy: {accuracy:.1%}"

        except Exception as e:
            return None, None, f"❌ Error during attack: {str(e)}"

    def generate_accuracy_graph(self):
        """Generate accuracy comparison graph"""
        if not self.accuracy_results:
            return None, "📊 No accuracy data available yet. Please run some extractions first."

        # Create figure
        fig, ax = plt.subplots(figsize=(10, 6))

        # Prepare data
        attacks = list(self.accuracy_results.keys())
        accuracies = [self.accuracy_results[attack] * 100 for attack in attacks]

        # Create colors based on accuracy
        colors = []
        for acc in accuracies:
            if acc >= 80:
                colors.append('#2ecc71')  # Green
            elif acc >= 60:
                colors.append('#f39c12')  # Orange
            else:
                colors.append('#e74c3c')  # Red

        # Create bars
        bars = ax.bar(attacks, accuracies, color=colors, alpha=0.8, edgecolor='black', linewidth=1)

        # Customize the plot
        ax.set_ylabel('Accuracy (%)', fontsize=12, fontweight='bold')
        ax.set_title('Watermark Extraction Accuracy Under Various Attacks',
                    fontsize=14, fontweight='bold', pad=20)
        ax.set_ylim(0, 100)

        # Add value labels on bars
        for bar, accuracy in zip(bars, accuracies):
            height = bar.get_height()
            ax.text(bar.get_x() + bar.get_width()/2., height + 1,
                   f'{accuracy:.1f}%', ha='center', va='bottom',
                   fontweight='bold', fontsize=10)

        # Add grid for better readability
        ax.grid(True, alpha=0.3, axis='y')
        ax.set_axisbelow(True)

        # Rotate x-axis labels for better fit
        plt.xticks(rotation=15, ha='right')
        plt.tight_layout()

        # Save the plot
        graph_path = "accuracy_graph.png"
        plt.savefig(graph_path, dpi=100, bbox_inches='tight')
        plt.close()

        return graph_path, "📊 Accuracy graph generated successfully!"

# Create system instance
watermark_system = WatermarkSystem()

# -------------------- Gradio Interface --------------------

with gr.Blocks(theme=gr.themes.Soft(), title="🌀 WaterMarkEye - Invisible Watermarking System") as demo:

    # Centered Title Section
    gr.Markdown(
        """
        <div style='text-align: center;'>
        <h1>🌀 WaterMarkEye - Invisible Watermarking System</h1>
        <h3>DWT-DCT Based Robust Digital Watermarking for Copyright Protection</h3>
        <p><strong>Protect your digital content with invisible, robust watermarks that survive common image processing attacks!</strong></p>
        </div>
        """,
        elem_id="centered-title"
    )

    with gr.Tabs():
        with gr.TabItem("📥 1. Upload Images"):
            with gr.Row():
                with gr.Column():
                    gr.Markdown("### 📷 Upload Cover Image")
                    cover_upload = gr.Image(label="Cover Image", type="numpy", height=300)
                    cover_status = gr.Textbox(label="Status", interactive=False)
                    cover_upload.upload(
                        watermark_system.upload_cover,
                        inputs=[cover_upload],
                        outputs=[cover_upload, cover_status]
                    )

                with gr.Column():
                    gr.Markdown("### 💧 Upload Logo/Watermark")
                    logo_upload = gr.Image(label="Logo Image", type="numpy", height=300)
                    logo_status = gr.Textbox(label="Status", interactive=False)
                    logo_upload.upload(
                        watermark_system.upload_logo,
                        inputs=[logo_upload],
                        outputs=[logo_upload, logo_status]
                    )

        with gr.TabItem("🔨 2. Embed Watermark"):
            with gr.Row():
                with gr.Column():
                    gr.Markdown("### 🎯 Embed Watermark")
                    embed_btn = gr.Button("🚀 Embed Watermark", variant="primary", size="lg")
                    embed_status = gr.Textbox(label="Embedding Status", interactive=False)

                with gr.Column():
                    gr.Markdown("### 💠 Watermarked Image")
                    watermarked_display = gr.Image(label="Watermarked Image", height=300, interactive=False)

            embed_btn.click(
                watermark_system.embed_watermark,
                outputs=[watermarked_display, embed_status]
            )

        with gr.TabItem("🔍 3. Extract Watermark (Clean)"):
            with gr.Row():
                with gr.Column():
                    gr.Markdown("### 🔍 Extract from Clean Image")
                    extract_clean_btn = gr.Button("✨ Extract Watermark", variant="primary", size="lg")
                    extract_clean_status = gr.Textbox(label="Extraction Status", interactive=False)

                with gr.Column():
                    gr.Markdown("### 📤 Extracted Watermark")
                    extracted_clean_display = gr.Image(label="Extracted Watermark", height=300, interactive=False)

            extract_clean_btn.click(
                watermark_system.extract_clean,
                outputs=[extracted_clean_display, extract_clean_status]
            )

        with gr.TabItem("🛡️ 4. Attack Tests"):
            gr.Markdown("### 🛡️ Test Watermark Robustness Against Attacks")

            with gr.Row():
                with gr.Column():
                    attack_type = gr.Dropdown(
                        choices=[
                            "JPEG Compression",
                            "Resizing",
                            "Gaussian Noise",
                            "Brightness",
                            "Blur",
                            "Contrast"
                        ],
                        label="Select Attack Type",
                        value="JPEG Compression"
                    )
                    attack_btn = gr.Button("⚡ Apply Attack & Extract", variant="primary", size="lg")
                    attack_status = gr.Textbox(label="Attack Status", interactive=False)

                with gr.Column():
                    gr.Markdown("### 🎯 Attacked Image")
                    attacked_display = gr.Image(label="After Attack", height=250, interactive=False)

                with gr.Column():
                    gr.Markdown("### 📤 Extracted After Attack")
                    extracted_attack_display = gr.Image(label="Extracted Watermark", height=250, interactive=False)

            attack_btn.click(
                watermark_system.apply_attack_and_extract,
                inputs=[attack_type],
                outputs=[attacked_display, extracted_attack_display, attack_status]
            )

        with gr.TabItem("📈 5. Accuracy Analysis"):
            gr.Markdown("### 📈 Watermark Extraction Accuracy Analysis")

            with gr.Row():
                with gr.Column():
                    gr.Markdown("""
                    ### 📊 Performance Metrics

                    This section shows the overall performance of the watermarking system across all tests.

                    **Accuracy Interpretation:**
                    - 🟢 **90-100%**: Excellent robustness
                    - 🟡 **70-89%**: Good robustness
                    - 🔴 **50-69%**: Moderate robustness
                    - 🔴 **<50%**: Needs improvement

                    **How to use:**
                    1. Run clean extraction and various attacks
                    2. Click "Generate Accuracy Graph" below
                    3. View overall system performance
                    """)

                    generate_graph_btn = gr.Button("📈 Generate Accuracy Graph", variant="primary", size="lg")
                    graph_status = gr.Textbox(label="Graph Status", interactive=False)

                with gr.Column():
                    gr.Markdown("### 📉 Accuracy Comparison Chart")
                    accuracy_graph = gr.Image(label="Accuracy Graph", height=400, interactive=False)

            generate_graph_btn.click(
                watermark_system.generate_accuracy_graph,
                outputs=[accuracy_graph, graph_status]
            )

        with gr.TabItem("📋 6. System Info"):
            gr.Markdown(
                """
                ## 🎯 System Overview

                **WaterMarkEye** implements a robust invisible watermarking system using **DWT-DCT Hybrid Transform**:

                ### 🔧 Technical Process:
                1. **Preprocessing**: Convert to YCbCr color space, extract Y (luminance) channel
                2. **DWT Transformation**: Apply 2-level Discrete Wavelet Transform using Haar wavelet
                3. **DCT Transformation**: Apply block-based Discrete Cosine Transform on LL subband
                4. **Embedding**: Insert watermark bits into mid-frequency DCT coefficients
                5. **Reconstruction**: Inverse DCT + Inverse DWT to reconstruct watermarked image
                6. **Detection**: Correlation-based extraction from same coefficients

                ### 🛡️ Robustness Features:
                - ✅ **Invisible**: Watermark is imperceptible to human vision
                - ✅ **Robust**: Survives common image processing attacks
                - ✅ **Secure**: Embedded in frequency domain for protection
                - ✅ **Reliable**: High extraction accuracy under various conditions

                ### 🎯 Supported Attacks:
                - **JPEG Compression** 📉 - Tests lossy compression resistance
                - **Resizing** 📏 - Tests scale invariance
                - **Gaussian Noise** 🎲 - Tests noise corruption resistance
                - **Brightness** 💡 - Tests illumination changes
                - **Blur** 🔍 - Tests smoothing filter resistance
                - **Contrast** 🌓 - Tests contrast changes

                ---
                *Built with ❤️ using Python, OpenCV, PyWavelets, and Gradio*
                """
            )

    # Footer
    gr.Markdown(
        """
        ---
        <div style='text-align: center'>
        <p>🌀 <b>WaterMarkEye</b> - Digital Copyright Protection System</p>
        <p><small>DWT-DCT Invisible Watermarking | Robust Image Authentication</small></p>
        </div>
        """
    )

# -------------------- Custom CSS for Centered Title --------------------

css = """
#centered-title {
    text-align: center !important;
}

#centered-title h1 {
    color: #2c3e50;
    margin-bottom: 10px;
}

#centered-title h3 {
    color: #7f8c8d;
    margin-bottom: 20px;
}
"""

demo.css = css

# -------------------- Launch Application --------------------

if __name__ == "__main__":
    demo.launch(share=True, debug=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://ac2632724a7a4e570f.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
